In [1]:
import cv2
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
from shapely.geometry import Polygon
import matplotlib.pyplot as plt

<h2>Funciones

In [2]:
def rellenar_hoyos(imagen_binaria):
    kernel = np.ones((5, 5), np.uint8)
    imagen_cerrada = cv2.morphologyEx(imagen_binaria, cv2.MORPH_CLOSE, kernel)
    return imagen_cerrada

In [3]:
def dibujar_y_rellenar_poligono(coordenadas1, coordenadas2, nombre, carpeta):
    img = np.zeros((112, 112, 3), np.uint8)

    puntos_ajustados1 = np.array([[(x, y) for x, y in coordenadas1]], np.int32)
    puntos_ajustados2 = np.array([[(x, y) for x, y in coordenadas2]], np.int32)

    cv2.fillPoly(img, [puntos_ajustados1], color=(255, 255, 255), lineType=cv2.LINE_AA)
    cv2.fillPoly(img, [puntos_ajustados2], color=(255, 255, 255), lineType=cv2.LINE_AA)

    _, imagen_binaria = cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 128, 255, cv2.THRESH_BINARY)

    imagen_con_hoyos_rellenados = rellenar_hoyos(imagen_binaria)
    
    cv2.imwrite("Mask/" + carpeta + "/" + nombre + ".jpg", imagen_con_hoyos_rellenados)
    print("Imagen guardada como " + nombre + ".jpg")

In [4]:
def save_frames(frame1, frame2, videoName, carpeta):
  vidcap = cv2.VideoCapture('Videos/'+videoName)
  success,image = vidcap.read()
  count = 0
  while success:
    if count == frame1 or count == frame2:
      cv2.imwrite("Frames/"+carpeta+"/frame"+str(count)+"_"+videoName[:-4]+".jpg", image) 
      print('Read frame', count, ': success')
    success,image = vidcap.read()
    count += 1

In [5]:
def get_coor(df):
  coor1 = []
  coor2 = []
  frame1 = df.Frame.iloc[0]
  
  for i in range(len(df.X1)):
    if frame1 == df.Frame.iloc[i]:
      coor1.append([df.X1.iloc[i],df.Y1.iloc[i]])
      coor1.append([df.X2.iloc[i],df.Y2.iloc[i]])
    else:
      coor2.append([df.X1.iloc[i],df.Y1.iloc[i]])
      coor2.append([df.X2.iloc[i],df.Y2.iloc[i]])
  return coor1, coor2

<h2>Lectura de los .csv

In [6]:
dfInfo = pd.read_csv('FileList.csv')
dfFrames = pd.read_csv('VolumeTracings.csv')

<h2>Coordenadas y frames del video

In [13]:
dfTrain = dfInfo[dfInfo.Split == "TRAIN"].copy()
for i in range(0,len(dfTrain)-1):
    print("Reading video "+str(dfTrain.FileName.iloc[i]))
    coor1, coor2 = get_coor(dfFrames[dfFrames.FileName == dfTrain.FileName.iloc[i]+'.avi'])
    coor1x = sorted(coor1, key=lambda coord: coord[0])
    coor1y = sorted(coor1, key=lambda coord: coord[1])

    coor2x = sorted(coor2, key=lambda coord: coord[0])
    coor2y = sorted(coor2, key=lambda coord: coord[1])

    frame1 = dfFrames[dfFrames.FileName == dfTrain.FileName.iloc[i]+'.avi'].Frame.iloc[0]
    l = len(dfFrames[dfFrames.FileName == dfTrain.FileName.iloc[i]+'.avi'])
    frame2 = dfFrames[dfFrames.FileName == dfTrain.FileName.iloc[i]+'.avi'].Frame.iloc[l-1]

    save_frames(frame1,frame2,dfTrain.FileName.iloc[i]+'.avi',"TRAIN")
    
    dibujar_y_rellenar_poligono(coor1x, coor1y, 'mask'+str(frame1)+"_"+dfTrain.FileName.iloc[i], "TRAIN")
    dibujar_y_rellenar_poligono(coor2x, coor2y, 'mask'+str(frame2)+"_"+dfTrain.FileName.iloc[i], "TRAIN")
    print(i,"\n")

Reading video 0X1002E8FBACD08477
Read frame 3 : success
Read frame 18 : success
Imagen guardada como mask3_0X1002E8FBACD08477.jpg
Imagen guardada como mask18_0X1002E8FBACD08477.jpg
0 

Reading video 0X1005D03EED19C65B
Read frame 24 : success
Read frame 35 : success
Imagen guardada como mask24_0X1005D03EED19C65B.jpg
Imagen guardada como mask35_0X1005D03EED19C65B.jpg
1 

Reading video 0X10075961BC11C88E
Read frame 91 : success
Read frame 108 : success
Imagen guardada como mask91_0X10075961BC11C88E.jpg
Imagen guardada como mask108_0X10075961BC11C88E.jpg
2 

Reading video 0X100E3B8D3280BEC5
Read frame 25 : success
Read frame 38 : success
Imagen guardada como mask25_0X100E3B8D3280BEC5.jpg
Imagen guardada como mask38_0X100E3B8D3280BEC5.jpg
3 

Reading video 0X100E491B3CD58DE2
Read frame 49 : success
Read frame 75 : success
Imagen guardada como mask49_0X100E491B3CD58DE2.jpg
Imagen guardada como mask75_0X100E491B3CD58DE2.jpg
4 

Reading video 0X100F044876B98F90
Read frame 56 : success
Read fra

IndexError: single positional indexer is out-of-bounds

In [12]:
dfTest = dfInfo[dfInfo.Split == "TEST"].copy()
for i in range(0,len(dfTest)-1):
    print("Reading video "+str(dfTest.FileName.iloc[i]))
    coor1, coor2 = get_coor(dfFrames[dfFrames.FileName == dfTest.FileName.iloc[i]+'.avi'])
    coor1x = sorted(coor1, key=lambda coord: coord[0])
    coor1y = sorted(coor1, key=lambda coord: coord[1])

    coor2x = sorted(coor2, key=lambda coord: coord[0])
    coor2y = sorted(coor2, key=lambda coord: coord[1])

    frame1 = dfFrames[dfFrames.FileName == dfTest.FileName.iloc[i]+'.avi'].Frame.iloc[0]
    l = len(dfFrames[dfFrames.FileName == dfTest.FileName.iloc[i]+'.avi'])
    frame2 = dfFrames[dfFrames.FileName == dfTest.FileName.iloc[i]+'.avi'].Frame.iloc[l-1]

    save_frames(frame1,frame2,dfTest.FileName.iloc[i]+'.avi',"TEST")
    
    dibujar_y_rellenar_poligono(coor1x, coor1y, 'mask'+str(frame1)+"_"+dfTest.FileName.iloc[i], "TEST")
    dibujar_y_rellenar_poligono(coor2x, coor2y, 'mask'+str(frame2)+"_"+dfTest.FileName.iloc[i], "TEST")
    print(i,"\n")

Reading video 0X100CF05D141FF143
Read frame 132 : success
Read frame 148 : success
Imagen guardada como mask132_0X100CF05D141FF143.jpg
Imagen guardada como mask148_0X100CF05D141FF143.jpg
0 

Reading video 0X1012703CDC1436FE
Read frame 154 : success
Read frame 172 : success
Imagen guardada como mask154_0X1012703CDC1436FE.jpg
Imagen guardada como mask172_0X1012703CDC1436FE.jpg
1 

Reading video 0X102CFB07F752AAE6
Read frame 163 : success
Read frame 184 : success
Imagen guardada como mask163_0X102CFB07F752AAE6.jpg
Imagen guardada como mask184_0X102CFB07F752AAE6.jpg
2 

Reading video 0X105B9EF57DE45DCB
Read frame 106 : success
Read frame 121 : success
Imagen guardada como mask106_0X105B9EF57DE45DCB.jpg
Imagen guardada como mask121_0X105B9EF57DE45DCB.jpg
3 

Reading video 0X10A28877E97DF540
Read frame 69 : success
Read frame 87 : success
Imagen guardada como mask69_0X10A28877E97DF540.jpg
Imagen guardada como mask87_0X10A28877E97DF540.jpg
4 

Reading video 0X10B7505562B0A702
Read frame 61 : 

In [11]:
dfVal = dfInfo[dfInfo.Split == "VAL"].copy()
for i in range(0,len(dfVal)-1):
    print("Reading video "+str(dfVal.FileName.iloc[i]))
    coor1, coor2 = get_coor(dfFrames[dfFrames.FileName == dfVal.FileName.iloc[i]+'.avi'])
    coor1x = sorted(coor1, key=lambda coord: coord[0])
    coor1y = sorted(coor1, key=lambda coord: coord[1])

    coor2x = sorted(coor2, key=lambda coord: coord[0])
    coor2y = sorted(coor2, key=lambda coord: coord[1])

    frame1 = dfFrames[dfFrames.FileName == dfVal.FileName.iloc[i]+'.avi'].Frame.iloc[0]
    l = len(dfFrames[dfFrames.FileName == dfVal.FileName.iloc[i]+'.avi'])
    frame2 = dfFrames[dfFrames.FileName == dfVal.FileName.iloc[i]+'.avi'].Frame.iloc[l-1]

    save_frames(frame1,frame2,dfVal.FileName.iloc[i]+'.avi',"VAL")
    
    dibujar_y_rellenar_poligono(coor1x, coor1y, 'mask'+str(frame1)+"_"+dfVal.FileName.iloc[i], "VAL")
    dibujar_y_rellenar_poligono(coor2x, coor2y, 'mask'+str(frame2)+"_"+dfVal.FileName.iloc[i], "VAL")
    print(i,"\n")

Reading video 0X100009310A3BD7FC
Read frame 46 : success
Read frame 61 : success
Imagen guardada como mask46_0X100009310A3BD7FC.jpg
Imagen guardada como mask61_0X100009310A3BD7FC.jpg
0 

Reading video 0X10094BA0A028EAC3
Read frame 137 : success
Read frame 156 : success
Imagen guardada como mask137_0X10094BA0A028EAC3.jpg
Imagen guardada como mask156_0X10094BA0A028EAC3.jpg
1 

Reading video 0X1013E8A4864781B
Read frame 35 : success
Read frame 46 : success
Imagen guardada como mask35_0X1013E8A4864781B.jpg
Imagen guardada como mask46_0X1013E8A4864781B.jpg
2 

Reading video 0X1039B49145DF4F25
Read frame 40 : success
Read frame 57 : success
Imagen guardada como mask40_0X1039B49145DF4F25.jpg
Imagen guardada como mask57_0X1039B49145DF4F25.jpg
3 

Reading video 0X105039B849CDB1CD
Read frame 55 : success
Read frame 73 : success
Imagen guardada como mask55_0X105039B849CDB1CD.jpg
Imagen guardada como mask73_0X105039B849CDB1CD.jpg
4 

Reading video 0X10ABB3401FC5EC2D
Read frame 51 : success
Read fr